In [3]:
from bs4 import BeautifulSoup
from requests import get

In [8]:
state = 'oregon'
cities = ['troutdale','portland','corvallis']
outputlist = []
for city in cities:
    url = 'https://www.bestplaces.net/voting/city/'+state+'/'+city
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    container = html_soup.find_all('div', class_ = 'col-md-12')
    output = container[1].find('b').text
    bad_chars = ['.', ','] 
    for i in bad_chars : 
        output = output.replace(i, '') 
    outputlist.append(output.split(' is ')[1].lower())

In [9]:
outputlist

['somewhat liberal', 'moderately liberal', 'moderately liberal']